In [43]:
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
#import scipy.spatial.distance as dist
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [44]:
def dtp(dist_mat):
    N, M = dist_mat.shape
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1,N+1):
        cost_mat[i, 0]  = np.inf
    for j in range(1, M+1):
        cost_mat[0, j] = np.inf
    traceback_mat = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            min_list = [cost_mat[i, j], # match = 0
                        cost_mat[i, j+1],   #insert = 1
                        cost_mat[i+1, j]]   # deletion = 2
            index_min = np.argmin(min_list)
            cost_mat[i+1,j+1] = dist_mat[i, j] + min_list[index_min]
            traceback_mat[i,j] = index_min 
    i = N-1
    j = M -1
    path = [(i,j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i,j]
        if tb_type == 0: # đi chéo
            i = i-1
            j = j-1
        elif tb_type == 1: # đi xuống
            i = i - 1
        elif tb_type == 2: # đi ngang
            j = j - 1
        path.append((i,j))
    cost_mat = cost_mat[1:, 1:]
    return path[::-1], cost_mat,traceback_mat


In [45]:
def distance_matrix(mat1, mat2):
    N = mat1.shape[0]
    M = mat2.shape[0]
    dist_mat = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            dist_mat[i, j] = np.linalg.norm(mat1[i, :] - mat2[j,:] )
    return dist_mat
    

In [46]:
def calculate_angle(a,b,c):
    a, b, c = np.array(a), np.array(b),  np.array(c)

    radians = np.arctan2(c[1]- b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180/np.pi)
    if angle > 180:
        angle = 360 - 180
    return angle

In [47]:
def extract_vid(vid_path, num):
    cap = cv2.VideoCapture(vid_path)
    N = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    matrix = []
    count = 0
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
        while(cap.isOpened()):
            try:
                ret, frame = cap.read()
                count += 1
            #Change from BGR to RGB
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
                img.flags.writeable = False

                #Make detection
                result = pose.process(img)

                # Back the original color channels
                img.flags.writeable = True
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                #Extract landmarks 
                try:
                    landmarks = result.pose_landmarks.landmark
                    #print(landmarks)
                except:
                    pass
                # Landmarks: là 1 list chứa tọa độ 33 điểm trên cơ thể
                theta = []
                for i in range(33):
                    for j in range(i+1, 33):
                        for k in range(j+1,33): 
                            a = [landmarks[i].x, landmarks[i].y]
                            b = [landmarks[j].x, landmarks[j].y]
                            c = [landmarks[k].x, landmarks[k].y]
                            angle = calculate_angle(a, b, c)
                            theta.append(angle)
                matrix.append(theta)

            except:
                pass
            if count == num:
                break
        
        return np.array(matrix), N



In [48]:
mat1,n = extract_vid("TestData/test6/Dumbell_headt.mp4",200)
mat1.shape

(200, 5456)

In [49]:
mat2, m= extract_vid("TestData/test6/Dumbell_headl.mp4",300)
mat2.shape

(300, 5456)

In [50]:
dist_mat = distance_matrix(mat1, mat2)
dist_mat

array([[4381.17326694, 4450.58071984, 4494.24309371, ..., 6800.98081   ,
        6798.2009517 , 6810.99059249],
       [4255.52189249, 4342.23431558, 4332.97164333, ..., 7104.65912743,
        7096.81157247, 7081.46198786],
       [4255.52189249, 4342.23431558, 4332.97164333, ..., 7104.65912743,
        7096.81157247, 7081.46198786],
       ...,
       [6766.67729555, 6808.05281007, 6845.17195386, ..., 4041.82523715,
        4028.54095735, 3968.03308547],
       [5998.89971033, 6006.28794541, 6040.07201479, ..., 5949.0686847 ,
        5937.64464486, 5884.30779095],
       [5843.10587283, 5838.50239961, 5879.1985852 , ..., 6247.66538075,
        6235.37003709, 6189.16869283]])

In [51]:
dist_mat.shape

(200, 300)

In [52]:
dist_mat[0,1]

4450.580719842527

In [53]:
path, cost_mat, tbm= dtp(dist_mat)
# print(cost_mat[n - 1, m - 1])
# print(cost_mat[n - 1, m - 1]/(n + m))

# plt.figure(figsize=(6, 4))
# plt.subplot(121)
# plt.title("Distance matrix")
# plt.imshow(dist_mat, cmap=plt.cm.binary, interpolation="nearest", origin="lower")
# plt.subplot(122)
# plt.title("Cost matrix")
# plt.imshow(cost_mat, cmap=plt.cm.binary, interpolation="nearest", origin="lower")
# x_path, y_path = zip(*path)
# plt.plot(y_path, x_path);

In [54]:
path[:50]


[(0, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 29),
 (2, 30),
 (3, 31),
 (4, 32),
 (5, 33),
 (6, 34),
 (7, 35),
 (7, 36),
 (7, 37),
 (7, 38),
 (7, 39),
 (7, 40),
 (7, 41),
 (7, 42),
 (7, 43),
 (7, 44),
 (7, 45),
 (7, 46),
 (7, 47),
 (7, 48),
 (7, 49)]

In [55]:
def save_video(vid_path, name_output):
    vid = cv2.VideoCapture(vid_path)
    frame_height = int(vid.get(4))
    frame_width = int(vid.get(3))
    frame_size = (frame_width, frame_height)
    fps = 20.0
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output = cv2.VideoWriter(name_output, fourcc, fps, frame_size)
    return output

In [56]:
cap1 = cv2.VideoCapture("TestData/test6/Dumbell_headt.mp4")
cap2 = cv2.VideoCapture("TestData/test6/Dumbell_headl.mp4")
output1 = save_video("TestData/test6/Dumbell_headt.mp4", 'OutputData/optest6/dbt.mp4')
output2 = save_video("TestData/test6/Dumbell_headl.mp4", 'OutputData/optest6/dbl.mp4')
stop = 0
while cap1.isOpened() or cap2.isOpened():
    stop += 1
    if stop == 2:
        break
    for i in path:
        cap1.set(1,i[0])
        cap2.set(1, i[1])  # Where frame_no is the frame you want
        okay1  , frame1 = cap1.read()
        okay2 , frame2 = cap2.read()
        
        if okay1:
            #hsv1 = cv2.cvtColor(frame1 , cv2.COLOR_BGR2HSV)
            cv2.imshow('trainer' , frame1)
            
            output1.write(frame1)
 
        if okay2:
            #hsv2 = cv2.cvtColor(frame2 , cv2.COLOR_BGR2HSV)
            cv2.imshow('user' , frame2)
            
            output2.write(frame2)

        if not okay1 or not okay2:
            print('Cant read the video , Exit!')
            break

        k = cv2.waitKey(20)
        if k == ord('q'):
            break 
        
        #cv2.waitKey(1)
    # if cv2.waitKey(1) and 0xFF == ord('q'):
    #     break
    cv2.waitKey(0)
cap1.release()
cap2.release()
output1.release()
output2.release()
cv2.destroyAllWindows()